In [66]:
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt

In [67]:
ENCODING_DICT = {
            'a': 0,
            'f': 1,
            'e': 2,
            'c': 3,
            'b': 4,
            'h': 5,
            'v': 6,
            'z': 7,
            '2': 8,
            'x': 9,
            'g': 10,
            'm': 11,
            'r': 12,
            'u': 13,
            'p': 14,
            's': 15,
            'd': 16,
            'n': 17,
            '6': 18,
            'k': 19,
            't': 20
            }

DECODING_DICT = {
            0: 'a',
            1: 'f',
            2: 'e',
            3: 'c',
            4: 'b',
            5: 'h',
            6: 'v',
            7: 'z',
            8: '2',
            9: 'x',
            10: 'g',
            11: 'm',
            12: 'r',
            13: 'u',
            14: 'p',
            15: 's',
            16: 'd',
            17: 'n',
            18: '6',
            19: 'k',
            20: 't'
            }

In [102]:
def imagepath_to_captcha(imagepath):
    return imagepath.split("_")[1].split(".")[0]

def to_onehot(captcha):
    label = np.zeros((len(captcha), len(ENCODING_DICT)))
    for index, char in enumerate(captcha):
        label[index, ENCODING_DICT[char]] = 1
    return tf.convert_to_tensor(label)

def captcha_from_onehot(tensor):
    label = []
    for index in tf.argmax(tensor, 1).numpy():
        label.append(DECODING_DICT[index])
    return "".join(label)

def load_dataset(filepath):
    filenames = os.listdir(filepath)
    x, y = [], []
    for file in filenames:
        imagepath = filepath + "/" + file
        image = tf.io.read_file(imagepath)
        image = tf.image.decode_image(image)
        pixels = tf.image.convert_image_dtype(image, tf.float32)
        # To show images: 
        # plt.imshow(tf.reshape(pixels, image.shape))
        # plt.title('Image as Pixels')
        # plt.show()
        x.append(pixels)
        captcha = imagepath_to_captcha(file).lower()
        y.append(to_onehot(captcha))
    return (tf.convert_to_tensor(x), tf.convert_to_tensor(y))


# Read input data

In [103]:
train_x, train_y = load_dataset("dataset/train")
test_x, test_y = load_dataset("dataset/test")

print("Training input shape:", train_x.shape)
print("Training output shape:", train_y.shape)
print("Testing input shape:", test_x.shape)
print("Testing output shape:", test_y.shape)

Training input shape: (8501, 50, 250, 3)
Training output shape: (8501, 6, 21)
Testing input shape: (1500, 50, 250, 3)
Testing output shape: (1500, 6, 21)


In [108]:
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(32, (2, 2), activation='relu', input_shape=(50, 250, 3)))
model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Conv2D(32, (2, 2), activation='relu'))
model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1008, activation='relu'))
model.add(keras.layers.Dense(504, activation='relu'))
model.add(keras.layers.Dense(252, activation='relu'))
model.add(keras.layers.Dense(126, activation='softmax'))
model.add(keras.layers.Reshape((6, 21)))


model.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_36 (Conv2D)          (None, 49, 249, 32)       416       
                                                                 
 max_pooling2d_35 (MaxPooli  (None, 24, 124, 32)       0         
 ng2D)                                                           
                                                                 
 conv2d_37 (Conv2D)          (None, 23, 123, 32)       4128      
                                                                 
 max_pooling2d_36 (MaxPooli  (None, 11, 61, 32)        0         
 ng2D)                                                           
                                                                 
 flatten_15 (Flatten)        (None, 21472)             0         
                                                                 
 dense_61 (Dense)            (None, 1008)            

In [109]:
model.compile(
    optimizer = "adam",
    loss = 'categorical_crossentropy'
)

In [113]:
history = model.fit(train_x, train_y,
                    epochs = 5)

print(history.history)

Epoch 1/5
125/266 [=============>................] - ETA: 1:03 - loss: 2.9101

In [100]:
y_pred = model.predict(test_x)

for i in range(len(y_pred)):
    print(captcha_from_onehot(test_y[i]), captcha_from_onehot(y_pred[i]))

16/16 [==============================] - 0s 20ms/step
ztasxg ekgcdp
tfxnze epnrxg
xkespv ebzf2x
uzrsav gdgzav
rfefnu ukgbkn
2z2tdt dkbnrh
rv6kvs 6abkbx
xeufz2 62mhtk
gfbkz2 sbkrxg
etpkmk aakhhv
ubkfpm 6bu6rk
2gcbup ebnt2g
gxhhku ebhh2f
xdubtu bphhuf
rddc2a gb2sbx
rtas2f ebntzb
6utvdr sxdkku
txdzdf gaukhe
2tmakk skugrn
zd2amf sbhnrc
6vedmk 6netxh
pe6ghs ekptr6
6grfkg 6zd6br
axsv2v ufphcx
usgssh ubp6rr
2ps2pc sdfbag
gvtmzs ektn6h
hhdbcg ebdcch
xkrczb ga66bk
knrchk trg6hn
mmzdfk mvz2hc
ttpcht tk2nrh
eeanxd uk2nrr
unadea sk2n6r
x2hkms sbxauc
u6prhm uz6esr
cgbcvr btngna
cfuu6s ebgubg
pvsnze rkxnce
e2pnfu gfzrfe
u6ts2z tabfbx
nfstua bedsdr
2knsnu 2du6ar
exczu2 eaunse
bseczv senfur
zbmctf 2bntfp
xv6xdz vuxfuz
c2pgre ebczrb
fuevfx huevs2
x2bfs6 ekssu6
2fhuua sbhrrb
nvrczm vdbsd6
afvpk2 ekngeb
gukdkr sbhbah
mgzepg g2kkhe
eubdkd gbxuxc
dfxxnh ergurn
rv6mpd amrkge
sgfnkc utfba6
xhvdzf adnb2d
zvdbkr 2bu6rr
kfk6fg adkghc
6rdctx mvukde
grzxzv adpbcu
stfuez dahp2b
dsscnt dducku
ssfxdv ebrx6g
uddtvh m